<a href="https://colab.research.google.com/github/Prakhar-porwal/local-voice-tts-clean/blob/main/notebooks/fast_tts_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Local Voice TTS - Turbo Mode (Google Colab)

This notebook runs the **Local Voice TTS Backend** on a **Free Google Colab T4 GPU**.
This is roughly **10x-20x faster** than the Hugging Face CPU Free Tier.

### Instructions:
1.  **Runtime** -> **Change runtime type** -> **T4 GPU** (should be default).
2.  **Runtime** -> **Run all**.
3.  Wait for the **Public URL** (ending in `ngrok-free.app` or similar) to appear at the bottom.
4.  Copy that URL and use it in your Vercel Frontend!

In [ ]:
# @title 1. Setup & Install (Final Fix)
import os

# 1. Clean previous runs
if os.path.exists("local-voice-tts-clean"):
    !rm -rf local-voice-tts-clean

# 2. Clone Repository
!git clone https://github.com/Prakhar-porwal/local-voice-tts-clean
%cd local-voice-tts-clean

# 3. System Dependencies
!apt-get -y install espeak-ng libsndfile1

# 4. Install Dependencies (Transformers 4.40.0 is CRITICAL for Colab)
!pip install transformers==4.40.0
!pip install "numpy<2.0" setuptools wheel

# 5. Install Other Deps
!pip install accelerate fastapi "uvicorn[standard]" librosa soundfile pydub python-multipart pyngrok nest_asyncio

# 6. Install TTS from Source (Community Fork)
print("Installing TTS...")
!pip install --no-build-isolation "git+https://github.com/idiap/coqui-ai-TTS.git#egg=coqui-tts"

print("\n✅ Setup Complete. Please RUN CELL 2 now.")


In [ ]:
# @title 2. Start Turbo Backend (Auto-Fix & Run)
import os
import threading
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# --- 1. AUTO-ACCEPT LICENSE TO PREVENT HANGING ---
os.environ["COQUI_TOS_AGREED"] = "1"

# --- 2. APPLY HOTFIX FOR 'isin_mps_friendly' ERROR ---
target_file = "/usr/local/lib/python3.12/dist-packages/TTS/tts/layers/tortoise/autoregressive.py"
if os.path.exists(target_file):
    with open(target_file, "r") as f:
        content = f.read()
    # Replace missing transformer function with torch equivalent
    if "isin_mps_friendly" in content:
        print("Applying Hotfix to TTS library...")
        new_content = content.replace(
            "from transformers.pytorch_utils import isin_mps_friendly as isin", 
            "from torch import isin"
        )
        with open(target_file, "w") as f:
            f.write(new_content)
        print("Hotfix applied! ✅")

# --- 3. SETUP NGROK ---
# FORCE KILL old tunnels to prevent "Already Online" error
ngrok.kill()

# SET YOUR TOKEN HERE
ngrok_token = "36jQzzt9HifzkJJKUwtGTOAt3nA_3hZTGahebR2E5xk4WDX7o"
ngrok.set_auth_token(ngrok_token)

nest_asyncio.apply()

try:
    public_url = ngrok.connect(7860).public_url
    print("\n================================================================")
    print(f"🚀 TURBO API URL: {public_url}")
    print("================================================================\n")
    print("Copy the URL above and paste it into your App settings!")
except Exception as e:
    print(f"Ngrok Error: {e}")
    # Try to get existing tunnel if connect failed
    tunnels = ngrok.get_tunnels()
    if tunnels:
        print(f"🚀 EXISTING URL: {tunnels[0].public_url}")

# --- 4. START SERVER ---
# Change directory to ensure we find main.py
os.chdir("/content/local-voice-tts-clean")
print("Starting Server in: " + os.getcwd())

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = uvicorn.Config("main:app", host="127.0.0.1", port=7860)
server = uvicorn.Server(config)
import asyncio
await server.serve()